In [ ]:
!pip install getdaft[aws,ray] --pre --extra-index-url https://pypi.anaconda.org/daft-nightly/simple
!pip install Pillow numpy

In [ ]:
CI = False

In [ ]:
USE_RAY = False

```{hint}
✨✨✨ **Run this notebook on Google Colab** ✨✨✨

You can [run this notebook yourself with Google Colab](https://colab.research.google.com/github/Eventual-Inc/Daft/blob/main/tutorials/image_querying/top_n_red_color.ipynb)!
```

# Top N Most Red Images

In this demo we will go through a simple example of using Daft to find the top N most red images out of the OpenImages dataset.

In [ ]:
###
# Settings for running on 10,000 rows in a distributed Ray cluster
###

if USE_RAY:
    import daft.context
    import ray

    # NOTE: Replace with the address to an existing running Ray cluster, or None to start a local Ray cluster
    RAY_CLUSTER_ADDRESS = "ray://localhost:10001"

    ray.init(
        address=RAY_CLUSTER_ADDRESS,
        runtime_env={"pip": ["getdaft", "pillow", "s3fs"]},
    )

    daft.context.set_runner_ray(address=RAY_CLUSTER_ADDRESS)


## Constructing our DataFrame

In [ ]:
import daft

df = daft.from_glob_path("s3://daft-public-data/open-images/validation-images/*")

if USE_RAY:
    df = df.limit(10000)
    df = df.repartition(64)
else:
    df = df.limit(100)

In [ ]:
df

### Filtering Data

In [ ]:
df = df.where(df["size"] < 300000)

**Daft is LAZY**: this filtering doesn't run immediately, but rather queues operations up in a query plan.

Now, let's define another filter with a lower bound for the image size

In [ ]:
df = df.where(df["size"] > 200000)

If we look at the plan, there are now two enqueued `Filter` operations!

In [ ]:
df.explain()

Doing these two `Filter`s one after another is really inefficient since we have to pass through the data twice!

Don't worry though - Daft's query optimizer will actually optimize this at runtime and merge the two `Filter` operations into one. You can view the optimized plan with `show_optimized=True`:

In [ ]:
df.explain(show_optimized=True)

This is just one example of query optimization, and Daft does many other really important ones such as Predicate Pushdowns and Column Pruning to keep your execution plans running efficiently.

Now we can **materialize** the filtered dataframe like so:

In [ ]:
# Materializes the dataframe and shows first 10 rows

df.collect()

Note that calling `.collect()` **materializes** the data. It will execute the above plan and all the computed data will be materialized in memory as long as the `df` variable is valid. This means that any subsequent operation on `df` will read from this materialized data instead of computing the entire plan again.

Let's prune the columns to just the "filepath" column, which is the only one we need at the moment:

In [ ]:
df = df.select("path")

In [ ]:
# Show doesn't materialize the data, but lets us peek at the first N rows
# produced by the current query plan

df.show(5)

### Working with Complex Data

Now let's do some data manipulation, starting with some simple ones (URLs) and finally images!

In [ ]:
df = df.with_column("image", df["path"].url.download())

In [ ]:
# Materialize the dataframe, so that we don't have to hit S3 again for subsequent operations
df.collect()

To decode the raw bytes we're downloading from the URL into an Image, we can simply use Daft's `.image.decode()` expression:

In [ ]:
df = df.with_column("image", df["image"].image.decode())

In [ ]:
df.show(5)

Sometimes you may need to run your own custom Python function. We can do this with the `.apply` Daft expression.

Let's run a custom `magic_red_detector` function, which will detect red pixels in our images and present us with a mask!

In [ ]:
import PIL
from PIL import ImageFilter
import numpy as np


def magic_red_detector(img: np.ndarray) -> PIL.Image.Image:
    """Gets a new image which is a mask covering all 'red' areas in the image"""
    img = PIL.Image.fromarray(img)
    lower = np.array([245, 100, 100])
    upper = np.array([10,255,255])
    lower_hue, upper_hue = lower[0, np.newaxis, np.newaxis], upper[0, np.newaxis, np.newaxis]
    lower_saturation_intensity, upper_saturation_intensity = lower[1:, np.newaxis, np.newaxis], upper[1:, np.newaxis, np.newaxis]    
    hsv = img.convert('HSV')
    hsv = np.asarray(hsv).T
    mask = np.all((hsv[1:, ...] >= lower_saturation_intensity) & (hsv[1:, ...] <= upper_saturation_intensity), axis=0) & ((hsv[0, ...] >= lower_hue) | (hsv[0, ...] <= upper_hue))
    img = PIL.Image.fromarray(mask.T)
    img = img.filter(ImageFilter.ModeFilter(size=5))
    return img


df = df.with_column(
    "red_mask",
    df["image"].apply(magic_red_detector, return_dtype=daft.DataType.python()),
)

In [ ]:
df.collect()

Now to determine how "red" an image is, we can simply sum up the number of pixels in the `red_mask` column!

In [ ]:
import numpy as np

def sum_mask(mask: PIL.Image.Image) -> int:
    val = np.asarray(mask).sum()
    return int(val)

df = df.with_column(
    "num_pixels_red",
    df["red_mask"].apply(sum_mask, return_dtype=daft.DataType.int64()),
)

In [ ]:
df.sort("num_pixels_red", desc=True).collect()